<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/Extract_Email_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pypdf2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00


In [6]:
!pip install tabulate


In [7]:
# Import necessary libraries
import os
import re
from tabulate import tabulate
import PyPDF2
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define path to the folder containing the PDFs
pdf_folder = '/content/drive/MyDrive/dataset Classification/Email'

# Extract text from PDFs
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Extract data using regular expressions
def extract_email_data(text):
    from_ = re.search(r'from:\s*(.*)', text, re.I)
    to_ = re.search(r'to:\s*(.*)', text, re.I)
    object_ = re.search(r'object:\s*(.*)', text, re.I)

    return [
        from_.group(1) if from_ else None,
        to_.group(1) if to_ else None,
        object_.group(1) if object_ else None
    ]

# Collecting data in a list
data = []

# Iterate over all PDFs in the folder and extract data
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        text = extract_text_from_pdf(pdf_path)
        email_data = extract_email_data(text)
        data.append(email_data)

# Display data using tabulate
headers = ["From", "To", "Object"]
print(tabulate(data, headers=headers, tablefmt='grid'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+-------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+----------+
| From                                                                                      | To                                                                                                | Object   |
+===========================================================================================+===================================================================================================+==========+
| Brandt, William                                                                           | Skillin, Anne; Catala, Louis V.; Lynch, Bill                                                      |          |
+--------------------------------------------------

## Named Entity Recognition (NER)

In [8]:
!pip install tabulate spacy
!python -m spacy download en_core_web_sm


2023-10-03 23:54:26.091045: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 23:54:26.981753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
# Import necessary libraries
import os
import spacy
from tabulate import tabulate
import PyPDF2
from google.colab import drive

# Load the SpaCy NER model
nlp = spacy.load('en_core_web_sm')

# Mount Google Drive
drive.mount('/content/drive')

# Define path to the folder containing the PDFs
pdf_folder = '/content/drive/My Drive/EA'

# Extract text from PDFs
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Extract email data using simple string parsing
def extract_email_data(text):
    # Split the document into individual emails based on 'From:'
    emails = text.split('From:')[1:]

    data = []
    for email in emails:
        from_ = email.split('To:')[0].strip()
        remaining = email.split('To:')[1]
        to_ = remaining.split('Subject:')[0].strip()
        subject_ = remaining.split('Subject:')[1].split('\n')[0].strip()
        data.append([from_, to_, subject_])

    return data

# Collecting data in a list
all_data = []

# Iterate over all PDFs in the folder and extract data
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        text = extract_text_from_pdf(pdf_path)
        email_data = extract_email_data(text)
        all_data.extend(email_data)

# Display data using tabulate
headers = ["From", "To", "Subject"]
print(tabulate(all_data, headers=headers, tablefmt='grid'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+---------------------------------------------+------------------------------------------------------------------------------+-------------------------------------------------------+
| From                                        | To                                                                           | Subject                                               |
+=============================================+==============================================================================+=======================================================+
| Fermandez, Henry L.                         | Hopkins Hadel, Linda L.                                                      | RE: KidsPeace                                         |
| Sent: Tuesday, September 25, 2001 5:23 PM   |                                                                              |             